In [1]:
import pathlib
import sys
import cv2
import torch
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Fix WindowsPath loading issue
if sys.platform != "win32":
    pathlib.WindowsPath = pathlib.PosixPath

# Load YOLOv5 model (runs on CPU/GPU as available)
yolo_model = torch.hub.load(
    'ultralytics/yolov5',
    'custom',
    path='E:/sugarcane_bud_classification/best.pt',
    force_reload=True
)

# Load MobileNet classification model
mobilenet_model = load_model(
    "E:/sugarcane_bud_classification/mobilenetv1_sugarcane_bud_classifier.h5"
)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\smsan/.cache\torch\hub\master.zip
YOLOv5  2025-6-6 Python-3.12.6 torch-2.4.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7053277 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


In [2]:
# Classification labels
class_names = ['Grade A', 'Grade B', 'Grade C']

# URL of the IP Webcam stream (replace with your stream URL)
ip_stream_url = 'http://192.168.225.88:8080/video'

# Open the video capture
cap = cv2.VideoCapture(ip_stream_url)
if not cap.isOpened():
    print(f"Error: Cannot open stream {ip_stream_url}")
    sys.exit(1)

print("Press 'q' to quit.")
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Convert BGR to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Run YOLOv5 detection
    results = yolo_model(rgb_frame)
    detections = results.pandas().xyxy[0]

    # Process each detection
    for _, row in detections.iterrows():
        x1, y1, x2, y2 = map(int, (row['xmin'], row['ymin'], row['xmax'], row['ymax']))

        # Crop and preprocess for classification
        crop = rgb_frame[y1:y2, x1:x2]
        if crop.size == 0:
            continue
        crop_resized = cv2.resize(crop, (224, 224))
        image_array = img_to_array(crop_resized)
        image_array = preprocess_input(image_array)
        image_array = np.expand_dims(image_array, axis=0)

        # Predict grade
        preds = mobilenet_model.predict(image_array, verbose=0)
        class_index = np.argmax(preds[0])
        label = class_names[class_index]

        # Draw bounding box and label on the original BGR frame
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
        font_scale = 1
        thickness = 2
        (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness)
        cv2.rectangle(frame, (x1, y1 - h - 10), (x1 + w, y1), (0, 255, 0), cv2.FILLED)
        cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 0), thickness)

    # Display the resulting frame
    cv2.imshow('Live Sugarcane Bud Grading', frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


Error: Cannot open stream http://192.168.225.88:8080/video


SystemExit: 1